In [452]:
import math
import numpy as np
np.float = float
np.int   = int
body_imu = [1.72, -15.86, -0.08]
crotch_imu = [1.86, -14.88, 0.57]
# body_imu = [-1.10, -1.36, 0.90]
# crotch_imu = [-1.01, -0.62, -14.12]
body_imu_rad    = np.deg2rad(body_imu)
crotch_imu_rad  = np.deg2rad(crotch_imu)
# --- helper 함수들 ---
#회전 행렬 만들기
#x축으로 roll, y축으로 pitch, z축으로 yaw
def rpy_to_mat(roll, pitch, yaw): 
    
    Rx = np.array([[1, 0, 0],
                   [0, math.cos(roll), -math.sin(roll)],
                   [0, math.sin(roll),  math.cos(roll)]])
    Ry = np.array([[ math.cos(pitch), 0, math.sin(pitch)],
                   [0, 1, 0],
                   [-math.sin(pitch), 0, math.cos(pitch)]])
    Rz = np.array([[math.cos(yaw), -math.sin(yaw), 0],
                   [math.sin(yaw),  math.cos(yaw), 0],
                   [0, 0, 1]])
    return Rz @ Ry @ Rx


def axis_angle_to_mat(axis, angle):
    axis = np.array(axis) / np.linalg.norm(axis) #단위 벡터로 만들기
    x,y,z = axis #축을 x,y,z로 나누기
    c, s = math.cos(angle), math.sin(angle) #삼각함수 각도 구하기
    C = 1-c #1-cos 저장
    # Rodrigues 회전 공식 적용
    return np.array([
      [x*x*C + c,   x*y*C - z*s, x*z*C + y*s],
      [y*x*C + z*s, y*y*C + c,   y*z*C - x*s],
      [z*x*C - y*s, z*y*C + x*s, z*z*C + c  ]
    ])

In [453]:
import json
from urdfpy import URDF

ROOT_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001'
# 1. URDF와 data.json 파일 경로
URDF_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1_inspire_hand_description/g1_29dof_rev_1_0_with_inspire_hand_FTP.urdf'    # URDF가 담긴 .txt 파일
DATA_PATH   = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'   # data.json 파일


finger_map = {'thumb':5, 'thumb2':4, 'index':3, 'middle':2, 'ring':1, 'pinky':0}

LEFTRIGHT = 'left'
WHICH_FINGER = 'index'
FRAME = 942

names = [LEFTRIGHT, 'shoulder', 'elbow', 'wrist', 'pitch', 'roll', 'yaw']   #얘는 list로 비교하면서 바꿨고 (아래 for i, joint in enumerate(joints) 부분에서)
handmap = {
                f'k{LEFTRIGHT.capitalize()}HandThumbRotation' : f'{LEFTRIGHT}_thumb_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandThumbBend' : f'{LEFTRIGHT}_thumb_3_joint',
                f'k{LEFTRIGHT.capitalize()}HandIndex' : f'{LEFTRIGHT}_index_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandMiddle' : f'{LEFTRIGHT}_middle_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandRing' : f'{LEFTRIGHT}_ring_1_joint',
                f'k{LEFTRIGHT.capitalize()}HandPinky' : f'{LEFTRIGHT}_little_1_joint',
            }

# 2. URDF 로드
robot = URDF.load(URDF_PATH)

# 3. data.json 로드
with open(DATA_PATH, 'r') as f:
    data = json.load(f)


joints  = data['info']['joint_names'][f'{LEFTRIGHT}_arm']  # 7개 joint names list       여기부터

for i, joint in enumerate(joints):
    tmp_name = ""

    joint = joint[1:]
    for name in names:
        find_res = joint.lower().find(name)
        if find_res == -1:
            continue
        else:
            tmp_name = tmp_name + joint[:find_res+len(name)] + '_'
            joint = joint[find_res+len(name):]
    
    joints[i] = (tmp_name[:-1] + '_joint').lower()

# 4. idx=0의 left_arm 관절각(qpos)과 joint 이름 목록 가져오기
qpos    = data['data'][FRAME]['states'][f'{LEFTRIGHT}_arm']['qpos']                         #여기까지는 arm을 뽑는부분

hand_qpos  = data['data'][FRAME]['states'][f'{LEFTRIGHT}_hand']['qpos']                 #여기부터
hand_joints = data['info']['joint_names'][f'{LEFTRIGHT}_hand']
hand_joints = [handmap[x] for x in hand_joints]                                       #여기까지는 hand joints 정리해두는부분 이중에 한개만 뽑아서 넣을거임

# 5. joint name → angle 매핑
q_dict = {name: angle for name, angle in zip(joints, qpos)}

q_dict[hand_joints[finger_map[WHICH_FINGER]]] = hand_qpos[finger_map[WHICH_FINGER]]       #hand q_dict에 추가

T_world_base = np.eye(4)
T_world_base[:3,:3] = rpy_to_mat(*body_imu_rad)
T_world_base[:3,3] = [0,0,crotch_imu_rad[2]]
# 6. forward kinematics 계산  
# ee_link = f'{LEFTRIGHT}_wrist_yaw_link'     # wrist뽑을때 uncomment, hand 뽑을때 comment
ee_link = hand_joints[finger_map[WHICH_FINGER]][:-6]                    # hand 뽑을때 uncomment, wrist 뽑을때 comment
print(ee_link)
# link_fk()는 각 링크별로 월드 기준 4×4 변환행렬을 반환
fk = robot.link_fk(q_dict)
# for link_obj, T in fk.items():
#     print(link_obj.name)
# T_ee_world = fk[ee_link]   # (4,4) 행렬
ee_link_obj  = robot.link_map[ee_link]    # 이름→Link 객체 얻기
T_ee_world   = fk[ee_link_obj]                 # 올바르게 인덱싱!

# 7. 위치(translation) 추출
pos_ee_world = T_world_base@T_ee_world 
print('Left arm end-effector position in world frame:', pos_ee_world)


left_index_1
Left arm end-effector position in world frame: [[ 0.49976709  0.25524336  0.82769782  0.26168679]
 [ 0.55512993 -0.82791925 -0.07987792  0.18907581]
 [ 0.66487865  0.49940019 -0.55546002  0.30870893]
 [ 0.          0.          0.          1.        ]]


In [454]:
# cam = robot['d435I_joint']
T_world_cam = np.eye(4)
T_world_cam[:3,:3] = rpy_to_mat(0, 0.8307767239493009, 0)
T_world_cam[:3, 3] = [0.0576235, 0.01753, 0.42987]   # :contentReference[oaicite:0]{index=0}


# 5) camera→left_wrist 변환
T_cam_ee = np.linalg.inv(T_world_cam) @ pos_ee_world

# 6) translation 성분이 바로 “카메라 기준” 손끝 좌표
pos_cam_ee = T_cam_ee[:3, 3]
print("Left wrist position in D435 camera frame:", pos_cam_ee)

Left wrist position in D435 camera frame: [0.2270724  0.17154581 0.06899243]


In [457]:
import cv2
import numpy as np

# 1) Intrinsics
width, height = 848, 480
fx = 426.7954
fy = 426.7954
# cx = width  / 2  # 424
# cy = height / 2  # 320
# fx=605.9278 
# fy=606.1917 
# cx = width  / 2  
# cy = height / 2  
cx = 414.0560
cy = 260.2649
# 2) pos_cam_ee: (X, Y, Z)  
#    X: 오른쪽(+), Y: 카메라 앞쪽(+), Z: 위쪽(+)
# print(pos_cam_ee)
# Xc =  pos_cam_ee[0]      
# Yc =  pos_cam_ee[1]      
# Zc =  pos_cam_ee[2]     
# Xc, Yc, Zc = T_cam_ee[:3,3]
X_urdf, Y_urdf, Z_urdf = pos_cam_ee

# 1) 축 재배열
Xc =  X_urdf
Yc = -Y_urdf
Zc = -Z_urdf
# 2) 투영
u = fx * (Yc / Xc) + cx
v = fy * (Zc / Xc) + cy

# 소수점→정수 픽셀 좌표
print(u,v)
pt = (int(round(u)), int(round(v)))
print(pt)
# 4) 이미지 읽고 표시  
img = cv2.imread("/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000942_color_0.jpg")
print(FRAME)
if img is None:
    raise RuntimeError("Unable to load image.")
cv2.circle(img, pt, 8, (255,0,0), thickness=-1)  # 빨간 점
cv2.putText(img, f"{pt}", (pt[0]+10, pt[1]),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

# 5) 결과 저장 (Jupyter에서 imshow 대신)
cv2.imwrite(f"projected_pointindex2{FRAME}.png", img)
print("Saved to projected_point.png")


91.62596439076509 130.58973241586406
(92, 131)
942
Saved to projected_point.png


In [471]:
import numpy as np
import json
import cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# =========================
# Paths / settings
# =========================
BODY_URDF_PATH = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
DATA_PATH      = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR        = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT        = '{:06d}_color_0.jpg'
OUT_DIR        = './ba_wrist_both'

FRAMES = [0,100,200,300,400,500,600,700,800,900,1000,1200,1300,1500,1600,942]

# Camera intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]], dtype=float)
dist = np.zeros(5)

# Body IMU (deg->rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# Optional: 초기 extrinsics (이전에 추정해 둔 값이 있다면 넣기)
R_cw_init = np.array([
    [-0.01675975, -0.96023504,  0.27868940],
    [-0.79857090, -0.15487018, -0.58163540],
    [ 0.60166737, -0.23230131, -0.76422017]
], dtype=float)
t_cw_init = np.array([0.01105951, 0.39473197, 0.37845197], dtype=float)

# =========================
# Utils
# =========================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    R,_ = cv2.Rodrigues(rvec.reshape(3,1))
    return R

def R_to_rodrigues(R):
    rvec,_ = cv2.Rodrigues(R)
    return rvec.flatten()

def click_point_optional(img_path, prompt, scale=1.0):
    """이미지에서 한 점을 클릭. 클릭 안하면 None 반환."""
    img = cv2.imread(img_path)
    if img is None:
        print(f"[WARN] image not found: {img_path}")
        return None, None
    h,w = img.shape[:2]
    disp = cv2.resize(img, (int(w*scale), int(h*scale)))
    pts = []
    def cb(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            pts.append((x/scale, y/scale))
            cv2.circle(disp, (int(x), int(y)), 6, (0,0,255), -1)
            cv2.imshow(prompt, disp)
    cv2.imshow(prompt, disp); cv2.setMouseCallback(prompt, cb)
    print(f"[{img_path}] {prompt}  (왼쪽 버튼 클릭: 선택 / 아무 키: 다음, 클릭 안하면 스킵)")
    cv2.waitKey(0); cv2.destroyWindow(prompt)
    if not pts:
        return None, img
    return pts[0], img

# =========================
# Load data
# =========================
robot = URDF.load(BODY_URDF_PATH)
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

# Joint maps (포괄적으로 키 커버)
l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
    'kLeftWristYaw'     :'left_wrist_yaw_joint',   # 대소문자 혼동 대비
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristyaw'     :'right_wrist_yaw_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}

def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = {}
    miss = []
    for n,v in zip(names, qpos):
        j = amap.get(n, None)
        if j is None:
            miss.append(n)
            continue
        q[j] = v
    if miss:
        print(f"[WARN][{side}][frame {frame}] unmapped joints skipped:", miss)

    fk = robot.link_fk(q)

    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)

    T_world_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    R_w = T_world_wrist[:3,:3]
    p_w = T_world_wrist[:3,3]
    return R_w, p_w

# =========================
# Collect correspondences
# =========================
samples = []  # each: dict with keys {side, frame, R_w, p_w, uv, img}
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

for fr in FRAMES:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    # 왼손
    try:
        RwL, pwL = wrist_pose_world(fr, 'left')
    except KeyError:
        RwL, pwL = None, None
        print(f"[WARN] FK failed for left, frame {fr}")
    if RwL is not None:
        uvL, img = click_point_optional(img_path, f'frame {fr} - LEFT wrist', scale=1.0)
        if uvL is not None:
            samples.append(dict(side='left', frame=fr, R_w=RwL, p_w=pwL, uv=np.array(uvL, float), img=img))
    else:
        img = cv2.imread(img_path)

    # 오른손
    try:
        RwR, pwR = wrist_pose_world(fr, 'right')
    except KeyError:
        RwR, pwR = None, None
        print(f"[WARN] FK failed for right, frame {fr}")
    if RwR is not None:
        uvR, _ = click_point_optional(img_path, f'frame {fr} - RIGHT wrist', scale=1.0)
        if uvR is not None:
            samples.append(dict(side='right', frame=fr, R_w=RwR, p_w=pwR, uv=np.array(uvR, float), img=img))

print(f"Collected samples: {len(samples)}")
if len(samples) < 6:
    raise RuntimeError("표본이 너무 적습니다. 최소 6개 이상 클릭해 주세요 (양손 합쳐).")

# =========================
# Bundle adjustment: solve for (rvec_cw, t_cw, d_L, d_R)
# =========================
# 초기값
rvec0 = R_to_rodrigues(R_cw_init)
t0    = t_cw_init.copy()
dL0   = np.zeros(3)   # 왼손목 오프셋 (wrist frame)
dR0   = np.zeros(3)   # 오른손목 오프셋 (wrist frame)
x0 = np.concatenate([rvec0, t0, dL0, dR0])  # 12 params

def residuals(x):
    rvec = x[0:3]
    t    = x[3:6]
    dL   = x[6:9]
    dR   = x[9:12]
    Rcw  = rodrigues_to_R(rvec)

    res = []
    for s in samples:
        d = dL if s['side']=='left' else dR
        # p_world = p_w + R_w * d_side
        p_world = s['p_w'] + s['R_w'] @ d
        p_cam = Rcw @ p_world + t
        X,Y,Z = p_cam
        if Z <= 1e-6:
            # 카메라 뒤쪽이면 큰 패널티
            res.extend([1000.0, 1000.0])
            continue
        u = fx * (X/Z) + cx
        v = fy * (Y/Z) + cy
        res.extend([u - s['uv'][0], v - s['uv'][1]])
    return np.array(res, dtype=float)

result = least_squares(
    residuals, x0, method='trf', loss='soft_l1', f_scale=3.0, max_nfev=200
)

x = result.x
rvec_est = x[0:3]
t_est    = x[3:6]
dL_est   = x[6:9]
dR_est   = x[9:12]
R_cw_est = rodrigues_to_R(rvec_est)

print("\n===== Joint optimization result =====")
print("R_cw (camera <- world):\n", R_cw_est)
print("t_cw:", t_est)
print("dL (left wrist offset in wrist frame) :", dL_est)
print("dR (right wrist offset in wrist frame):", dR_est)
print("cost:", result.cost, "  #sum(residual^2)/2")
print("status:", result.status, result.message)

# 평균 재투영 오차
res = residuals(x).reshape(-1,2)
per_point_err = np.linalg.norm(res, axis=1)
print("Reprojection error mean(px):", per_point_err.mean(), "  std(px):", per_point_err.std())

# =========================
# Visualization
# =========================
def proj_point_world(p_world):
    p_cam = R_cw_est @ p_world + t_est
    X,Y,Z = p_cam
    if Z <= 1e-6: return None
    u = fx*(X/Z)+cx; v = fy*(Y/Z)+cy
    return int(round(u)), int(round(v))

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

for s in samples:
    img = s['img'].copy()
    d = dL_est if s['side']=='left' else dR_est
    p_world = s['p_w'] + s['R_w'] @ d
    uv_est = proj_point_world(p_world)

    # GT click
    cv2.circle(img, tuple(s['uv'].astype(int)), 7, (0,0,255), -1)
    if uv_est is not None:
        cv2.circle(img, uv_est, 7, (255,0,0), -1)
        cv2.line(img, tuple(s['uv'].astype(int)), uv_est, (0,255,255), 2)
        err = np.linalg.norm(np.array(uv_est, float) - s['uv'])
    else:
        err = 999.0

    label = f"{s['side']}  f{ s['frame'] }  err={err:.2f}px"
    cv2.putText(img, label, (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)

    out_path = str(Path(OUT_DIR) / f"ba_{s['side']}_{s['frame']:06d}.png")
    cv2.imwrite(out_path, img)
    print("Saved:", out_path)


[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg] frame 0 - LEFT wrist  (왼쪽 버튼 클릭: 선택 / 아무 키: 다음, 클릭 안하면 스킵)
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg] frame 0 - RIGHT wrist  (왼쪽 버튼 클릭: 선택 / 아무 키: 다음, 클릭 안하면 스킵)
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000100_color_0.jpg] frame 100 - LEFT wrist  (왼쪽 버튼 클릭: 선택 / 아무 키: 다음, 클릭 안하면 스킵)
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000100_color_0.jpg] frame 100 - RIGHT wrist  (왼쪽 버튼 클릭: 선택 / 아무 키: 다음, 클릭 안하면 스킵)
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000200_color_0.jpg] frame 200 - LEFT wrist  (왼쪽 버튼 클릭: 선택 / 아무 키: 다음, 클릭 안하면 스킵)
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000

In [474]:
import numpy as np
import json, cv2
from pathlib import Path
from urdfpy import URDF

# ====== 경로/설정 ======
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'
OUT_DIR         = './test_vis_wrist_tip'

# 테스트할 프레임
FRAMES = [1, 10, 60, 94, 121, 384, 487, 597, 612, 781, 813, 942, 1205, 1398, 1456, 1589, 1678]

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649

# ====== 방금 추정된 Extrinsics / Offsets ======
R_cw = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

dL = np.array([-0.01958220,  0.03467778, -0.00843096])  # left wrist local
dR = np.array([ 0.00669406, -0.00187255,  0.03843392])  # right wrist local

# 로봇 IMU (deg->rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# ====== 유틸 ======
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def project_world(p_world):
    p_cam = R_cw @ p_world + t_cw
    X, Y, Z = p_cam
    if Z <= 1e-6:
        return None, p_cam
    u = fx * (X/Z) + cx
    v = fy * (Y/Z) + cy
    return (int(round(u)), int(round(v))), p_cam

# ====== 로드 ======
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print("[WARN] right hand URDF load failed:", e)

with open(DATA_PATH, 'r') as f:
    data = json.load(f)

l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
    'kLeftWristYaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristyaw'     :'right_wrist_yaw_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
# 오른손 손가락 매핑은 실제 URDF 이름에 맞게 수정 필요
r_hand_map = {}  # 오른손 tip 투영 안 쓰면 비워둠

def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = {}
    for n,v in zip(names, qpos):
        j = amap.get(n)
        if j is not None: q[j] = v
    fk = robot.link_fk(q)

    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_world_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    R_w = T_world_wrist[:3,:3]
    p_w = T_world_wrist[:3,3]
    return R_w, p_w

def left_index_tip_world(frame):
    # 왼손목
    Rw, pw = wrist_pose_world(frame, 'left')
    # 왼손 q
    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = { l_hand_map[n]: v for n,v in zip(names, qpos) }
    fk_h = left_hand.link_fk(qh)
    T_wrist_tip = fk_h[ left_hand.link_map['L_index_tip'] ]
    # 손목 오프셋 보정(dL)
    T_world_base = np.eye(4)
    T_world_base[:3,:3] = Rw
    T_world_base[:3,3]  = pw
    # 오프셋을 손목 로컬에서 적용 후 손 모델 변환
    T_off = np.eye(4); T_off[:3,3] = dL
    T_world_tip = T_world_base @ T_off @ T_wrist_tip
    return T_world_tip[:3,3]

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

for fr in FRAMES:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print("[WARN] missing image:", img_path)
        continue

    # 왼/오 손목
    RwL, pwL = wrist_pose_world(fr, 'left')
    RwR, pwR = wrist_pose_world(fr, 'right')
    p_wrist_L = pwL + RwL @ dL
    p_wrist_R = pwR + RwR @ dR

    uv_L, pcam_L = project_world(p_wrist_L)
    uv_R, pcam_R = project_world(p_wrist_R)

    # 왼 index tip
    p_tip_L = left_index_tip_world(fr)
    uv_tip_L, _ = project_world(p_tip_L)

    vis = img.copy()
    if uv_L:
        cv2.circle(vis, uv_L, 8, (0,255,255), -1)
        cv2.putText(vis, 'LW', (uv_L[0]+6, uv_L[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    if uv_R:
        cv2.circle(vis, uv_R, 8, (255,255,0), -1)
        cv2.putText(vis, 'RW', (uv_R[0]+6, uv_R[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
    if uv_tip_L:
        cv2.circle(vis, uv_tip_L, 7, (0,0,255), -1)
        cv2.putText(vis, 'L_tip', (uv_tip_L[0]+6, uv_tip_L[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)
    out_path = str(Path(OUT_DIR) / f'vis_{fr:06d}.png')
    cv2.imwrite(out_path, vis)
    print("Saved:", out_path)


Saved: test_vis_wrist_tip/vis_000001.png
Saved: test_vis_wrist_tip/vis_000010.png
Saved: test_vis_wrist_tip/vis_000060.png
Saved: test_vis_wrist_tip/vis_000094.png
Saved: test_vis_wrist_tip/vis_000121.png
Saved: test_vis_wrist_tip/vis_000384.png
Saved: test_vis_wrist_tip/vis_000487.png
Saved: test_vis_wrist_tip/vis_000597.png
Saved: test_vis_wrist_tip/vis_000612.png
Saved: test_vis_wrist_tip/vis_000781.png
Saved: test_vis_wrist_tip/vis_000813.png
Saved: test_vis_wrist_tip/vis_000942.png
Saved: test_vis_wrist_tip/vis_001205.png
Saved: test_vis_wrist_tip/vis_001398.png
Saved: test_vis_wrist_tip/vis_001456.png
Saved: test_vis_wrist_tip/vis_001589.png
Saved: test_vis_wrist_tip/vis_001678.png


In [475]:
def click_point(img_path, prompt='click', scale=1.0):
    img = cv2.imread(img_path); h,w = img.shape[:2]
    disp = cv2.resize(img, (int(w*scale), int(h*scale)))
    pts = []
    def cb(e,x,y,flags,param):
        if e==cv2.EVENT_LBUTTONDOWN:
            pts.append((x/scale, y/scale))
            cv2.circle(disp, (int(x),int(y)), 6, (0,0,255), -1)
            cv2.imshow(prompt, disp)
    cv2.imshow(prompt, disp); cv2.setMouseCallback(prompt, cb)
    print(f'[{img_path}] {prompt}: 왼쪽 클릭 후 키 입력')
    cv2.waitKey(0); cv2.destroyWindow(prompt)
    return (np.array(pts[0],float) if pts else None), img

TEST_FRAMES = [942, 600, 1300]   # 원하는 프레임
for fr in TEST_FRAMES:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    (uv_click_L, img) = click_point(img_path, f'frame {fr} - click LEFT wrist')
    RwL, pwL = wrist_pose_world(fr, 'left')
    p_wrist_L = pwL + RwL @ dL
    uv_pred_L, _ = project_world(p_wrist_L)

    vis = img.copy()
    if uv_click_L is not None and uv_pred_L is not None:
        err = np.linalg.norm(uv_click_L - np.array(uv_pred_L, float))
        cv2.circle(vis, tuple(uv_click_L.astype(int)), 7, (0,0,255), -1)
        cv2.circle(vis, uv_pred_L, 7, (255,0,0), -1)
        cv2.line(vis, tuple(uv_click_L.astype(int)), uv_pred_L, (0,255,255), 2)
        cv2.putText(vis, f'err={err:.2f}px', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0), 2)
        print(f'frame {fr}: LEFT wrist error = {err:.2f} px')
    out_path = str(Path(OUT_DIR) / f'err_left_{fr:06d}.png')
    cv2.imwrite(out_path, vis)
    print("Saved:", out_path)


[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000942_color_0.jpg] frame 942 - click LEFT wrist: 왼쪽 클릭 후 키 입력
Saved: test_vis_wrist_tip/err_left_000942.png
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000600_color_0.jpg] frame 600 - click LEFT wrist: 왼쪽 클릭 후 키 입력
Saved: test_vis_wrist_tip/err_left_000600.png
[/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/001300_color_0.jpg] frame 1300 - click LEFT wrist: 왼쪽 클릭 후 키 입력
Saved: test_vis_wrist_tip/err_left_001300.png


In [476]:
import numpy as np
import json
import cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 또는 None
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'
OUT_DIR         = './tip_vis'

# 시각화할 프레임들
FRAMES_VIS = [0,100,200,300,400,500,600,900,1300,1500,1600,942]

# (선택) 손끝 미세보정에 사용할 프레임들 — 손끝이 화면에 잘 보이는 프레임만 남기세요
DO_REFINE = False
FRAMES_TIP = [942, 500, 600]   # 예시. 필요에 따라 수정

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]], dtype=float)
dist = np.zeros(5)

# ===== 공동최적화 결과(카메라 ← 월드) =====
R_cw = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 (손목 로컬 좌표계에서)
dL = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)   # left
dR = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)   # right

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    K = np.array([[0, -k[2], k[1]],
                  [k[2], 0, -k[0]],
                  [-k[1], k[0], 0]])
    R = np.eye(3) + np.sin(theta)*K + (1-np.cos(theta))*(K@K)
    return R

def project_world(p_world):
    """world 3D → pixel, with check"""
    p_cam = R_cw @ p_world + t_cw
    X, Y, Z = p_cam
    if Z <= 0:
        return None, p_cam
    u = fx * (X / Z) + cx
    v = fy * (Y / Z) + cy
    return (int(round(u)), int(round(v))), p_cam

def click_point(img_path, win='click', scale=1.0):
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(img_path)
    h, w = img.shape[:2]
    disp = cv2.resize(img, (int(w*scale), int(h*scale)))
    pts = []
    def cb(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            pts.append((x/scale, y/scale))
            cv2.circle(disp, (int(x), int(y)), 6, (0,0,255), -1)
            cv2.imshow(win, disp)
    cv2.imshow(win, disp); cv2.setMouseCallback(win, cb)
    print(f'[{img_path}] 손끝 픽셀을 클릭하세요. (아무 키로 다음)')
    cv2.waitKey(0); cv2.destroyWindow(win)
    if not pts:
        raise RuntimeError("클릭되지 않았습니다.")
    return np.array(pts[0], dtype=float), img

# =========================================================
# 데이터/URDF 로드 및 맵
# =========================================================
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print(f"[WARN] RIGHT_HAND_URDF 로드 실패: {e}. 오른손 손끝은 생략합니다.")
        right_hand = None

with open(DATA_PATH, 'r') as f:
    data = json.load(f)

l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
# 오른손 키 이름은 실제 data['info']['joint_names']['right_hand'] 확인 후 맞춰주세요.
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

def safe_qdict(names_list, qpos_list, name_to_joint):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in name_to_joint:
            q[name_to_joint[n]] = v
        else:
            # 매핑이 없으면 무시 (경고만)
            print(f"[WARN] hand joint map missing key: {n}")
    return q

# =========================================================
# BODY URDF에서 wrist→hand 장착 변환 자동 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    # 왼손/오른손 구분이 있으면 우선순위
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    return picked, picked.origin

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')

if T_wrist_to_hand_L is None:
    print("[WARN] left wrist->hand mount not found. Use Identity.")
    T_wrist_to_hand_L = np.eye(4)
else:
    print("[MOUNT-LEFT] parent:", jL.parent, "child:", jL.child)
    print(T_wrist_to_hand_L)

if T_wrist_to_hand_R is None:
    print("[WARN] right wrist->hand mount not found. Use Identity.")
    T_wrist_to_hand_R = np.eye(4)
else:
    print("[MOUNT-RIGHT] parent:", jR.parent, "child:", jR.child)
    print(T_wrist_to_hand_R)

# =========================================================
# FK
# =========================================================
def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)
    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off=None, dRvec=None, dT=None):
    if d_off is None: d_off = dL
    if dRvec is None: dRvec = np.zeros(3)
    if dT is None: dT = np.zeros(3)

    Rw, pw = wrist_pose_world(frame, 'left')
    # wrist local offset
    T_off = np.eye(4); T_off[:3,3] = d_off
    # mount from BODY urdf + small correction
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    # hand joints
    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_qdict(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    T_hand_tip = fk_h[left_hand.link_map['L_index_tip']]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off=None, dRvec=None, dT=None):
    if right_hand is None:
        return None
    if d_off is None: d_off = dR
    if dRvec is None: dRvec = np.zeros(3)
    if dT is None: dT = np.zeros(3)

    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off

    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_qdict(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)

    # 오른손 URDF에서의 tip 링크명 확인 필요
    if 'R_index_tip' not in right_hand.link_map:
        print("[WARN] 'R_index_tip' not in right_hand.link_map. Skip.")
        return None

    T_hand_tip = fk_h[right_hand.link_map['R_index_tip']]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

# =========================================================
# (선택) 손끝 장착변환 미세 보정
# =========================================================
def refine_mount(side='left', frames=None, scale=1.0):
    """
    side: 'left' or 'right'
    frames: 클릭할 프레임 리스트
    결과: dRvec, dT (mount 보정) 반환
    """
    if frames is None or len(frames) == 0:
        print("[refine] no frames. skip.")
        return np.zeros(3), np.zeros(3)

    uv_list = []
    pw_list = []

    for fr in frames:
        img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
        try:
            uv, _ = click_point(img_path, scale=scale)
        except Exception as e:
            print(f"[refine] frame {fr} click fail: {e}")
            continue

        if side == 'left':
            p_w = left_index_tip_world(fr)  # 기본값(dL, mount 그대로)로 계산한 world 점
        else:
            p_w = right_index_tip_world(fr)

        if p_w is None:
            print(f"[refine] frame {fr}: tip world is None. skip.")
            continue

        uv_list.append(uv)
        pw_list.append(p_w)

    if len(pw_list) < 3:
        print(f"[refine] collected {len(pw_list)} pairs (<3). skip.")
        return np.zeros(3), np.zeros(3)

    uv = np.asarray(uv_list, dtype=float)
    Pw = np.asarray(pw_list, dtype=float)  # (N,3)

    # 파라미터: [rx, ry, rz, tx, ty, tz]
    x0 = np.zeros(6)

    def residual(x):
        rvec = x[:3]
        dT   = x[3:]
        errs = []
        for i in range(len(Pw)):
            p_w = Pw[i]
            # mount 보정 적용해서 다시 tip world 좌표를 만들면 좋지만,
            # 여기서는 p_w를 고정(이미 base mount로 계산된 world)하고,
            # 카메라 투영만 보정하는 방식은 맞지 않음.
            # -> 정확히 하려면 frame별로 다시 FK해야 한다.
            # 간단히 하기 위해 frame별 재계산으로 변경:
            # 프레임 번호를 유지하도록 조금 수정한다.

        return np.array(errs)

    # 위의 간단함수를 위해 프레임별 다시 FK하도록 재구성
    pairs = []
    for fr, uv_gt in zip(frames, uv_list):
        pairs.append((fr, uv_gt))

    def residual_fk(x):
        rvec = x[:3]
        dT   = x[3:]
        errs = []
        for (fr, uv_gt) in pairs:
            if side == 'left':
                p_w = left_index_tip_world(fr, d_off=(dL if side=='left' else dR),
                                           dRvec=rvec, dT=dT)
            else:
                p_w = right_index_tip_world(fr, d_off=(dR if side=='right' else dL),
                                            dRvec=rvec, dT=dT)
            if p_w is None:
                continue
            p_cam = R_cw @ p_w + t_cw
            X,Y,Z = p_cam
            if Z <= 0:
                # 큰 페널티
                errs.extend([1000.0, 1000.0])
                continue
            u = fx*X/Z + cx
            v = fy*Y/Z + cy
            errs.extend([u - uv_gt[0], v - uv_gt[1]])
        return np.array(errs)

    res = least_squares(residual_fk, x0, verbose=1, ftol=1e-8, xtol=1e-8, gtol=1e-8,
                        loss='soft_l1', f_scale=3.0, max_nfev=200)
    dRvec = res.x[:3]
    dT    = res.x[3:]
    print("=== refine result ({}) ===".format(side))
    print("dRvec:", dRvec, "  |dR| deg =", np.linalg.norm(dRvec)*180/np.pi)
    print("dT:", dT, " (m)")
    print("cost:", res.cost, "status:", res.status, res.message)

    # 최종 평균/표준편차 출력
    errs = residual_fk(res.x).reshape(-1,2)
    e2 = np.linalg.norm(errs, axis=1)
    print("Reproj err mean/std (px):", e2.mean(), e2.std())
    return dRvec, dT

# =========================================================
# 시각화 루프
# =========================================================
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# (선택) tip 보정
dRvec_L = np.zeros(3); dT_L = np.zeros(3)
dRvec_R = np.zeros(3); dT_R = np.zeros(3)
if DO_REFINE:
    print("\n[Refine LEFT tip mount]")
    dRvec_L, dT_L = refine_mount('left', frames=FRAMES_TIP, scale=1.0)
    if right_hand is not None:
        print("\n[Refine RIGHT tip mount]")
        dRvec_R, dT_R = refine_mount('right', frames=FRAMES_TIP, scale=1.0)

for fr in FRAMES_VIS:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print(f"[WARN] image not found: {img_path}")
        continue

    # 왼/오 손목
    RwL, pwL = wrist_pose_world(fr, 'left')
    uv_wL, pc_wL = project_world(pwL)

    RwR, pwR = wrist_pose_world(fr, 'right')
    uv_wR, pc_wR = project_world(pwR)

    # 왼/오 index tip
    p_tip_L = left_index_tip_world(fr, d_off=dL, dRvec=dRvec_L, dT=dT_L)
    uv_tL, pc_tL = project_world(p_tip_L) if p_tip_L is not None else (None, None)

    p_tip_R = right_index_tip_world(fr, d_off=dR, dRvec=dRvec_R, dT=dT_R)
    uv_tR, pc_tR = project_world(p_tip_R) if p_tip_R is not None else (None, None)

    vis = img.copy()

    # 손목
    if uv_wL is not None:
        cv2.circle(vis, uv_wL, 8, (0,255,255), -1)
        cv2.putText(vis, "LW", (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    if uv_wR is not None:
        cv2.circle(vis, uv_wR, 8, (255,255,0), -1)
        cv2.putText(vis, "RW", (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)

    # 손끝
    if uv_tL is not None:
        cv2.circle(vis, uv_tL, 7, (0,0,255), -1)
        cv2.putText(vis, "L_tip", (uv_tL[0]+6, uv_tL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    if uv_tR is not None:
        cv2.circle(vis, uv_tR, 7, (255,0,0), -1)
        cv2.putText(vis, "R_tip", (uv_tR[0]+6, uv_tR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)

    out_path = str(Path(OUT_DIR) / f'vis_tip_{fr}.png')
    cv2.imwrite(out_path, vis)
    print(f"Saved: {out_path}")

print("\n완료!\n- mount 추출 로그 확인\n- 손목이 맞는데 손끝이 어긋나면 DO_REFINE=True로 바꿔서 몇 프레임 클릭 후 dRvec/dT 결과를 적용해봐.")


[MOUNT-LEFT] parent: left_wrist_yaw_link child: left_hand_palm_link
[[ 1.      0.      0.      0.0415]
 [ 0.      1.      0.      0.003 ]
 [-0.      0.      1.      0.    ]
 [ 0.      0.      0.      1.    ]]
[MOUNT-RIGHT] parent: right_wrist_yaw_link child: right_hand_palm_link
[[ 1.      0.      0.      0.0415]
 [ 0.      1.      0.     -0.003 ]
 [-0.      0.      1.      0.    ]
 [ 0.      0.      0.      1.    ]]
Saved: tip_vis/vis_tip_0.png
Saved: tip_vis/vis_tip_100.png
Saved: tip_vis/vis_tip_200.png
Saved: tip_vis/vis_tip_300.png
Saved: tip_vis/vis_tip_400.png
Saved: tip_vis/vis_tip_500.png
Saved: tip_vis/vis_tip_600.png
Saved: tip_vis/vis_tip_900.png
Saved: tip_vis/vis_tip_1300.png
Saved: tip_vis/vis_tip_1500.png
Saved: tip_vis/vis_tip_1600.png
Saved: tip_vis/vis_tip_942.png

완료!
- mount 추출 로그 확인
- 손목이 맞는데 손끝이 어긋나면 DO_REFINE=True로 바꿔서 몇 프레임 클릭 후 dRvec/dT 결과를 적용해봐.


In [482]:
import numpy as np
import json
import cv2
from pathlib import Path
from urdfpy import URDF
from scipy.optimize import least_squares

# =========================================================
# 경로 / 설정
# =========================================================
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 또는 None
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'
OUT_DIR         = './joint_opt_vis'

# ======= 캘리브/시각화 프레임 =======
FRAMES_CALIB = [0,100,200,300,400,500,600,700,800,900,1000,1200,1300,1400,1500,1600,942]
FRAMES_VIS   = FRAMES_CALIB

# 카메라 intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0,  0,  1]], dtype=float)
dist = np.zeros(5)

# ===== 초기 Extrinsics (camera ← world) : 이전 공동최적화 결과값을 초기치로 사용 =====
R_cw_init = np.array([
    [ 0.22643170, -0.96805671, -0.10767957],
    [-0.36084943,  0.01931217, -0.93242411],
    [ 0.90471895,  0.24998648, -0.34494983]
], dtype=float)
t_cw_init = np.array([-0.01571018, 0.35198288, 0.29489303], dtype=float)

# 손목 오프셋 초기값 (손목 로컬 좌표계)
dL_init = np.array([-0.01958220,  0.03467778, -0.00843096], dtype=float)   # left
dR_init = np.array([ 0.00669406, -0.00187255,  0.03843392], dtype=float)   # right

# 로봇 바디 IMU (deg→rad)
body_imu = np.deg2rad([1.72, -15.86, -0.08])  # roll, pitch, yaw

# 가중치 (손목 2D > 손끝 2D)
W_WRIST = 1.0
W_TIP   = 0.7

# =========================================================
# 유틸
# =========================================================
def rpy_to_mat(r, p, y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    Kc = np.array([[0, -k[2], k[1]],
                   [k[2], 0, -k[0]],
                   [-k[1], k[0], 0]])
    return np.eye(3) + np.sin(theta)*Kc + (1-np.cos(theta))*(Kc@Kc)

def R_to_rodrigues(R):
    rvec, _ = cv2.Rodrigues(R.astype(np.float64))
    return rvec.flatten()

def to_h(T_or_R, t=None):
    if T_or_R.shape == (4,4):
        return T_or_R.copy()
    T = np.eye(4)
    T[:3,:3] = T_or_R
    if t is not None:
        T[:3,3] = t
    return T

# 클릭 도구 ------------------------------------------------
class ClickCollector:
    def __init__(self, scale=1.0):
        self.scale = scale
        self.pt = None

    def _cb(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.pt = (x/self.scale, y/self.scale)
            disp = param
            cv2.circle(disp, (x,y), 6, (0,0,255), -1)
            cv2.imshow(self.win, disp)

    def click_or_skip(self, img_path, label):
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(img_path)
        h, w = img.shape[:2]
        disp = cv2.resize(img, (int(w*self.scale), int(h*self.scale)))
        self.pt = None
        self.win = f'click_{label}'
        cv2.imshow(self.win, disp)
        cv2.setMouseCallback(self.win, self._cb, disp)
        print(f'[ {label} ]  {img_path}  — 클릭(좌클릭) 또는 s키로 건너뛰기')
        while True:
            k = cv2.waitKey(0) & 0xFF
            if k in (27, ord('s')):  # ESC or 's' -> skip
                cv2.destroyWindow(self.win)
                return None
            if self.pt is not None:
                cv2.destroyWindow(self.win)
                return np.array(self.pt, dtype=float)
            # otherwise keep waiting

# =========================================================
# 데이터/URDF 로드 및 맵
# =========================================================
print('[load] URDFs ...')
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print(f"[WARN] RIGHT_HAND_URDF 로드 실패: {e}. 오른손 손끝은 생략합니다.")
        right_hand = None

print('[load] data.json ...')
with open(DATA_PATH, 'r') as f:
    data = json.load(f)

l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
# 오른손 키 이름은 실제 data['info']['joint_names']['right_hand'] 확인 후 맞춰주세요.
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

def safe_qdict(names_list, qpos_list, name_to_joint):
    q = {}
    for n, v in zip(names_list, qpos_list):
        if n in name_to_joint:
            q[name_to_joint[n]] = v
        else:
            # 경고만 출력하고 무시
            pass
    return q

# =========================================================
# BODY URDF에서 wrist→hand 장착 변환 자동 추출
# =========================================================
def find_mount_transform(robot, wrist_link, prefer_side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            child_low = j.child.lower()
            if ('hand' in child_low) or ('finger' in child_low):
                cand.append(j)
    if not cand:
        return None, None
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            if prefer_side == 'left' and ('l_' in j.child.lower() or 'left' in j.child.lower()):
                picked = j; break
            if prefer_side == 'right' and ('r_' in j.child.lower() or 'right' in j.child.lower()):
                picked = j; break
    # urdfpy Joint.origin 은 4x4 ndarray
    return picked, picked.origin

jL, T_wrist_to_hand_L = find_mount_transform(robot, 'left_wrist_yaw_link', 'left')
jR, T_wrist_to_hand_R = find_mount_transform(robot, 'right_wrist_yaw_link', 'right')

if T_wrist_to_hand_L is None:
    print('[WARN] left wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_L = np.eye(4)
else:
    print('[MOUNT-LEFT] parent:', jL.parent, 'child:', jL.child)
    # print(T_wrist_to_hand_L)

if T_wrist_to_hand_R is None:
    print('[WARN] right wrist->hand mount not found. Use Identity.')
    T_wrist_to_hand_R = np.eye(4)
else:
    print('[MOUNT-RIGHT] parent:', jR.parent, 'child:', jR.child)
    # print(T_wrist_to_hand_R)

# =========================================================
# FK 모듈
# =========================================================
def wrist_pose_world(frame, side='left'):
    if side == 'left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap  = l_arm_map
        wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap  = r_arm_map
        wrist_link = 'right_wrist_yaw_link'

    q = { amap[n]: v for n,v in zip(names, qpos) if n in amap }
    fk = robot.link_fk(q)
    T_world_base = np.eye(4)
    T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    Rw = T_wrist[:3,:3]
    pw = T_wrist[:3,3]
    return Rw, pw

def left_index_tip_world(frame, d_off, dRvec, dT):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_qdict(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    T_hand_tip = fk_h[left_hand.link_map['L_index_tip']]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

def right_index_tip_world(frame, d_off, dRvec, dT):
    if right_hand is None:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = d_off
    R_delta = rodrigues_to_R(dRvec)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_qdict(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    if 'R_index_tip' not in right_hand.link_map:
        return None
    T_hand_tip = fk_h[right_hand.link_map['R_index_tip']]

    T_world_wrist = np.eye(4); T_world_wrist[:3,:3]=Rw; T_world_wrist[:3,3]=pw
    T_world_tip = T_world_wrist @ T_off @ T_mount @ T_hand_tip
    return T_world_tip[:3,3]

# =========================================================
# 클릭 수집 (왼손목, 오른손목, 왼손끝, 오른손끝)
# =========================================================
collector = ClickCollector(scale=1.0)

obs = []  # 리스트 of dict per frame: {'frame':f,'LW':uv or None,'RW':..., 'LT':..., 'RT':...}
for fr in FRAMES_CALIB:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    print(f"\n[frame {fr}] 수집 시작")
    entry = {'frame': fr, 'LW': None, 'RW': None, 'LT': None, 'RT': None}
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-LW(wrist-left)')
        entry['LW'] = uv
    except Exception as e:
        print('[skip LW]', e)
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-RW(wrist-right)')
        entry['RW'] = uv
    except Exception as e:
        print('[skip RW]', e)
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-LT(left index tip)')
        entry['LT'] = uv
    except Exception as e:
        print('[skip LT]', e)
    try:
        uv = collector.click_or_skip(img_path, f'{fr}-RT(right index tip)')
        entry['RT'] = uv
    except Exception as e:
        print('[skip RT]', e)
    obs.append(entry)

valid_cnt = sum( (e['LW'] is not None) or (e['RW'] is not None) or (e['LT'] is not None) or (e['RT'] is not None) for e in obs )
print(f"\n[collect] frames with any clicks: {valid_cnt}/{len(obs)}")
if valid_cnt < 4:
    raise RuntimeError('클릭 데이터가 너무 적습니다. 최소 4프레임 이상 권장')

# =========================================================
# 최적화 변수 구성
# x = [ rvec_cw(3), t_cw(3), dL(3), dR(3), rvec_L(3), dT_L(3), rvec_R(3), dT_R(3) ] => 24
# =========================================================
rvec_cw0 = R_to_rodrigues(R_cw_init)
x0 = np.concatenate([
    rvec_cw0, t_cw_init,
    dL_init, dR_init,
    np.zeros(3), np.zeros(3),  # rvec_L, dT_L
    np.zeros(3), np.zeros(3)   # rvec_R, dT_R
], axis=0)

# bounds (느슨하게)
# 각 회전은 약 ±30deg, 오프셋은 ±5cm, mount translation은 ±3cm
rot_bound = np.deg2rad(30.0)
trans_wrist_bound = 0.05
trans_mount_bound = 0.03
lb = np.array([
    -rot_bound, -rot_bound, -rot_bound,   # rvec_cw
    -1.0, -1.0, -1.0,                     # t_cw (m) — 넓게 허용
    -trans_wrist_bound, -trans_wrist_bound, -trans_wrist_bound,   # dL
    -trans_wrist_bound, -trans_wrist_bound, -trans_wrist_bound,   # dR
    -rot_bound, -rot_bound, -rot_bound,   # rvec_L
    -trans_mount_bound, -trans_mount_bound, -trans_mount_bound,   # dT_L
    -rot_bound, -rot_bound, -rot_bound,   # rvec_R
    -trans_mount_bound, -trans_mount_bound, -trans_mount_bound,   # dT_R
])
ub = -lb

# 잔차 함수 ------------------------------------------------
def residual(x):
    rvec_cw = x[0:3]
    t_cw    = x[3:6]
    dL      = x[6:9]
    dR      = x[9:12]
    rvec_L  = x[12:15]
    dT_L    = x[15:18]
    rvec_R  = x[18:21]
    dT_R    = x[21:24]

    R_cw = rodrigues_to_R(rvec_cw)
    R_cw_cv, _ = cv2.Rodrigues(R_cw.astype(np.float64))  # not used, keep compatibility

    res = []
    for e in obs:
        fr = e['frame']
        # 손목 좌표 (world)
        RwL, pwL = wrist_pose_world(fr, 'left')
        RwR, pwR = wrist_pose_world(fr, 'right')
        # wrist offset 적용 (로컬)
        pL = (RwL @ dL) + pwL
        pR = (RwR @ dR) + pwR

        # 투영 helper
        def proj(pw):
            pc = R_cw @ pw + t_cw
            X,Y,Z = pc
            if Z <= 0:
                return None
            u = fx*X/Z + cx
            v = fy*Y/Z + cy
            return np.array([u, v], dtype=float)

        # 손목 residual
        if e['LW'] is not None:
            uv = proj(pL)
            if uv is None:
                res += [1e3, 1e3]
            else:
                res += list(W_WRIST * (uv - e['LW']))
        if e['RW'] is not None:
            uv = proj(pR)
            if uv is None:
                res += [1e3, 1e3]
            else:
                res += list(W_WRIST * (uv - e['RW']))

        # 손끝 residual
        if e['LT'] is not None:
            p_tip_L = left_index_tip_world(fr, dL, rvec_L, dT_L)
            uv = proj(p_tip_L)
            if uv is None:
                res += [1e3, 1e3]
            else:
                res += list(W_TIP * (uv - e['LT']))
        if e['RT'] is not None:
            p_tip_R = right_index_tip_world(fr, dR, rvec_R, dT_R)
            if p_tip_R is not None:
                uv = proj(p_tip_R)
                if uv is None:
                    res += [1e3, 1e3]
                else:
                    res += list(W_TIP * (uv - e['RT']))
            # 오른손 URDF 미구현이면 잔차를 넣지 않음
    return np.array(res, dtype=float)

print('\n[opt] start least_squares ...')
res = least_squares(
    residual, x0, verbose=2,
    ftol=1e-9, xtol=1e-9, gtol=1e-9, max_nfev=400,
    loss='soft_l1', f_scale=3.0
)
print('\n===== Joint optimization result =====')
print('status:', res.status, res.message)
print('cost:', res.cost)

x = res.x
rvec_cw = x[0:3]; t_cw = x[3:6]
dL = x[6:9]; dR = x[9:12]
rvec_L = x[12:15]; dT_L = x[15:18]
rvec_R = x[18:21]; dT_R = x[21:24]
R_cw = rodrigues_to_R(rvec_cw)
print('R_cw:\n', R_cw)
print('t_cw:', t_cw)
print('dL:', dL)
print('dR:', dR)
print('dRvec_L:', rvec_L, '  dT_L:', dT_L)
print('dRvec_R:', rvec_R, '  dT_R:', dT_R)

# 최종 평균 reprojection error 보고
r = residual(x).reshape(-1,2)
err = np.linalg.norm(r, axis=1)
print('Reprojection error mean(px):', err.mean(), ' std(px):', err.std())

# =========================================================
# 시각화 저장
# =========================================================
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def project_point(R_cw, t_cw, pw):
    pc = R_cw @ pw + t_cw
    X,Y,Z = pc
    if Z <= 0:
        return None, pc
    u = fx*X/Z + cx
    v = fy*Y/Z + cy
    return (int(round(u)), int(round(v))), pc

for fr in FRAMES_VIS:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print('[WARN] image not found:', img_path)
        continue
    vis = img.copy()

    RwL, pwL = wrist_pose_world(fr, 'left')
    RwR, pwR = wrist_pose_world(fr, 'right')
    p_wL = (RwL @ dL) + pwL
    p_wR = (RwR @ dR) + pwR

    uv_wL, pc_wL = project_point(R_cw, t_cw, p_wL)
    uv_wR, pc_wR = project_point(R_cw, t_cw, p_wR)

    p_tL = left_index_tip_world(fr, dL, rvec_L, dT_L)
    uv_tL, pc_tL = project_point(R_cw, t_cw, p_tL) if p_tL is not None else (None, None)

    p_tR = right_index_tip_world(fr, dR, rvec_R, dT_R)
    uv_tR, pc_tR = project_point(R_cw, t_cw, p_tR) if p_tR is not None else (None, None)

    if uv_wL is not None:
        cv2.circle(vis, uv_wL, 8, (0,255,255), -1); cv2.putText(vis, 'LW', (uv_wL[0]+6, uv_wL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
    if uv_wR is not None:
        cv2.circle(vis, uv_wR, 8, (255,255,0), -1); cv2.putText(vis, 'RW', (uv_wR[0]+6, uv_wR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,0), 2)
    if uv_tL is not None:
        cv2.circle(vis, uv_tL, 7, (0,0,255), -1); cv2.putText(vis, 'L_tip', (uv_tL[0]+6, uv_tL[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    if uv_tR is not None:
        cv2.circle(vis, uv_tR, 7, (255,0,0), -1); cv2.putText(vis, 'R_tip', (uv_tR[0]+6, uv_tR[1]-6), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
    out_path = str(Path(OUT_DIR) / f'vis_{fr}.png')
    cv2.imwrite(out_path, vis)
    print('Saved:', out_path)

print('\n완료!\n- 클릭 수가 많을수록(특히 손목+손끝 둘 다) 결과가 안정적입니다.'
      '\n- 오른손 손끝이 비어 있으면 RT 항은 자동으로 생략됩니다.'
      '\n- 결과가 과도하게 흔들리면 가중치(W_WRIST/W_TIP)나 bounds를 조절해 주세요.')

PARAM_PATH = './joint_opt_params.json'
params = {
    "R_cw": R_cw.tolist(),
    "t_cw": t_cw.tolist(),
    "dL":   dL.tolist(),
    "dR":   dR.tolist(),
    "rvec_L": rvec_L.tolist(),
    "dT_L":   dT_L.tolist(),
    "rvec_R": rvec_R.tolist(),
    "dT_R":   dT_R.tolist(),
}
with open(PARAM_PATH, 'w') as f:
    json.dump(params, f, indent=2)
print("Saved params to", PARAM_PATH)


[load] URDFs ...
[load] data.json ...
[MOUNT-LEFT] parent: left_wrist_yaw_link child: left_hand_palm_link
[MOUNT-RIGHT] parent: right_wrist_yaw_link child: right_hand_palm_link

[frame 0] 수집 시작
[ 0-LW(wrist-left) ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-RW(wrist-right) ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-LT(left index tip) ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기
[ 0-RT(right index tip) ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000000_color_0.jpg  — 클릭(좌클릭) 또는 s키로 건너뛰기

[frame 100] 수집 시작
[ 100-LW(wrist-left) ]  /home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors/000100_

In [483]:
import numpy as np
import json, cv2
from pathlib import Path
from urdfpy import URDF

# ---------------- paths ----------------
BODY_URDF_PATH  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/g1/g1_body29_hand14.urdf'
LEFT_HAND_URDF  = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_left.urdf'
RIGHT_HAND_URDF = '/home/scilab/Documents/teleoperation/avp_teleoperate/assets/inspire_hand/inspire_hand_right.urdf'  # 없으면 '' 가능
DATA_PATH       = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/data.json'
IMG_DIR         = '/home/scilab/Documents/teleoperation/avp_teleoperate/teleop/utils/datanalysis/episode_0001/colors'
IMG_FMT         = '{:06d}_color_0.jpg'

PARAM_PATH      = './joint_opt_params.json'   # 1단계에서 저장한 파일
OUT_DIR_TEST    = './test_vis'

# 테스트할 프레임들
FRAMES_TEST = [50, 75, 125, 225, 275, 325, 425, 525, 625, 725, 825, 925, 1025, 1125, 1225, 1325, 1425, 1525, 1625]

# camera intrinsics
fx, fy = 605.9278, 606.1917
cx, cy = 414.0560, 260.2649

# IMU
body_imu = np.deg2rad([1.72, -15.86, -0.08])

def rpy_to_mat(r,p,y):
    Rx = np.array([[1,0,0],[0,np.cos(r),-np.sin(r)],[0,np.sin(r),np.cos(r)]])
    Ry = np.array([[np.cos(p),0,np.sin(p)],[0,1,0],[-np.sin(p),0,np.cos(p)]])
    Rz = np.array([[np.cos(y),-np.sin(y),0],[np.sin(y),np.cos(y),0],[0,0,1]])
    return Rz @ Ry @ Rx

def rodrigues_to_R(rvec):
    theta = np.linalg.norm(rvec)
    if theta < 1e-12:
        return np.eye(3)
    k = rvec / theta
    K = np.array([[0,-k[2],k[1]],[k[2],0,-k[0]],[-k[1],k[0],0]])
    return np.eye(3) + np.sin(theta)*K + (1-np.cos(theta))*(K@K)

# ---- load params ----
with open(PARAM_PATH,'r') as f:
    P = json.load(f)
R_cw = np.array(P['R_cw'], dtype=float)
t_cw = np.array(P['t_cw'], dtype=float)
dL   = np.array(P['dL'],   dtype=float)
dR   = np.array(P['dR'],   dtype=float)
rvec_L = np.array(P['rvec_L'], dtype=float)
dT_L   = np.array(P['dT_L'],   dtype=float)
rvec_R = np.array(P['rvec_R'], dtype=float)
dT_R   = np.array(P['dT_R'],   dtype=float)

# ---- load URDFs ----
robot = URDF.load(BODY_URDF_PATH)
left_hand = URDF.load(LEFT_HAND_URDF)
right_hand = None
if RIGHT_HAND_URDF:
    try:
        right_hand = URDF.load(RIGHT_HAND_URDF)
    except Exception as e:
        print('[WARN] right hand urdf load fail:', e)

# joint maps
l_arm_map = {
    'kLeftShoulderPitch':'left_shoulder_pitch_joint',
    'kLeftShoulderRoll' :'left_shoulder_roll_joint',
    'kLeftShoulderYaw'  :'left_shoulder_yaw_joint',
    'kLeftElbow'        :'left_elbow_joint',
    'kLeftWristRoll'    :'left_wrist_roll_joint',
    'kLeftWristPitch'   :'left_wrist_pitch_joint',
    'kLeftWristyaw'     :'left_wrist_yaw_joint',
}
r_arm_map = {
    'kRightShoulderPitch':'right_shoulder_pitch_joint',
    'kRightShoulderRoll' :'right_shoulder_roll_joint',
    'kRightShoulderYaw'  :'right_shoulder_yaw_joint',
    'kRightElbow'        :'right_elbow_joint',
    'kRightWristRoll'    :'right_wrist_roll_joint',
    'kRightWristPitch'   :'right_wrist_pitch_joint',
    'kRightWristYaw'     :'right_wrist_yaw_joint',
}
l_hand_map = {
    'kLeftHandPinky':'L_pinky_proximal_joint',
    'kLeftHandRing' :'L_ring_proximal_joint',
    'kLeftHandMiddle':'L_middle_proximal_joint',
    'kLeftHandIndex':'L_index_proximal_joint',
    'kLeftHandThumbBend':'L_thumb_proximal_pitch_joint',
    'kLeftHandThumbRotation':'L_thumb_proximal_yaw_joint',
}
r_hand_map = {
    'kRightHandPinky':'R_pinky_proximal_joint',
    'kRightHandRing' :'R_ring_proximal_joint',
    'kRightHandMiddle':'R_middle_proximal_joint',
    'kRightHandIndex':'R_index_proximal_joint',
    'kRightHandThumbBend':'R_thumb_proximal_pitch_joint',
    'kRightHandThumbRotation':'R_thumb_proximal_yaw_joint',
}

with open(DATA_PATH,'r') as f:
    data = json.load(f)

def safe_qdict(names, qpos, mapping):
    out = {}
    for n,v in zip(names, qpos):
        if n in mapping:
            out[mapping[n]] = v
    return out

# mount transform from BODY urdf
def find_mount(robot, wrist_link, side='left'):
    cand = []
    for j in robot.joints:
        if j.parent == wrist_link:
            c = j.child.lower()
            if 'hand' in c or 'finger' in c:
                cand.append(j)
    if not cand:
        return np.eye(4)
    picked = cand[0]
    if len(cand) > 1:
        for j in cand:
            c = j.child.lower()
            if side=='left' and ('l_' in c or 'left' in c): picked = j; break
            if side=='right' and ('r_' in c or 'right' in c): picked = j; break
    return picked.origin

T_wrist_to_hand_L = find_mount(robot, 'left_wrist_yaw_link', 'left')
T_wrist_to_hand_R = find_mount(robot, 'right_wrist_yaw_link', 'right')

def wrist_pose_world(frame, side='left'):
    if side=='left':
        names = data['info']['joint_names']['left_arm']
        qpos  = data['data'][frame]['states']['left_arm']['qpos']
        amap = l_arm_map; wrist_link = 'left_wrist_yaw_link'
    else:
        names = data['info']['joint_names']['right_arm']
        qpos  = data['data'][frame]['states']['right_arm']['qpos']
        amap = r_arm_map; wrist_link = 'right_wrist_yaw_link'
    q = safe_qdict(names, qpos, amap)
    fk = robot.link_fk(q)
    T_world_base = np.eye(4); T_world_base[:3,:3] = rpy_to_mat(*body_imu)
    T_wrist = T_world_base @ fk[ robot.link_map[wrist_link] ]
    return T_wrist[:3,:3], T_wrist[:3,3]

def left_tip_world(frame):
    Rw, pw = wrist_pose_world(frame, 'left')
    T_off = np.eye(4); T_off[:3,3] = dL
    R_delta = rodrigues_to_R(rvec_L)
    T_mount = T_wrist_to_hand_L.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT_L

    names = data['info']['joint_names']['left_hand']
    qpos  = data['data'][frame]['states']['left_hand']['qpos']
    qh = safe_qdict(names, qpos, l_hand_map)
    fk_h = left_hand.link_fk(qh)
    T_tip = fk_h[left_hand.link_map['L_index_tip']]

    T_w = np.eye(4); T_w[:3,:3]=Rw; T_w[:3,3]=pw
    Tw_tip = T_w @ T_off @ T_mount @ T_tip
    return Tw_tip[:3,3]

def right_tip_world(frame):
    if right_hand is None or 'R_index_tip' not in right_hand.link_map:
        return None
    Rw, pw = wrist_pose_world(frame, 'right')
    T_off = np.eye(4); T_off[:3,3] = dR
    R_delta = rodrigues_to_R(rvec_R)
    T_mount = T_wrist_to_hand_R.copy()
    T_mount[:3,:3] = R_delta @ T_mount[:3,:3]
    T_mount[:3,3] += dT_R

    names = data['info']['joint_names']['right_hand']
    qpos  = data['data'][frame]['states']['right_hand']['qpos']
    qh = safe_qdict(names, qpos, r_hand_map)
    fk_h = right_hand.link_fk(qh)
    T_tip = fk_h[right_hand.link_map['R_index_tip']]

    T_w = np.eye(4); T_w[:3,:3]=Rw; T_w[:3,3]=pw
    Tw_tip = T_w @ T_off @ T_mount @ T_tip
    return Tw_tip[:3,3]

def project(R_cw, t_cw, pw):
    pc = R_cw @ pw + t_cw
    X,Y,Z = pc
    if Z <= 0: return None
    u = fx*X/Z + cx; v = fy*Y/Z + cy
    return int(round(u)), int(round(v))

Path(OUT_DIR_TEST).mkdir(parents=True, exist_ok=True)

for fr in FRAMES_TEST:
    img_path = str(Path(IMG_DIR) / IMG_FMT.format(fr))
    img = cv2.imread(img_path)
    if img is None:
        print('[skip] image not found:', img_path); continue
    vis = img.copy()

    RwL, pwL = wrist_pose_world(fr, 'left')
    RwR, pwR = wrist_pose_world(fr, 'right')

    p_wL = pwL + RwL @ dL
    p_wR = pwR + RwR @ dR

    uv_wL = project(R_cw, t_cw, p_wL)
    uv_wR = project(R_cw, t_cw, p_wR)

    p_tL = left_tip_world(fr)
    uv_tL = project(R_cw, t_cw, p_tL) if p_tL is not None else None

    p_tR = right_tip_world(fr)
    uv_tR = project(R_cw, t_cw, p_tR) if p_tR is not None else None

    if uv_wL: cv2.circle(vis, uv_wL, 8, (0,255,255), -1); cv2.putText(vis,'LW',(uv_wL[0]+6,uv_wL[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,255,255),2)
    if uv_wR: cv2.circle(vis, uv_wR, 8, (255,255,0), -1); cv2.putText(vis,'RW',(uv_wR[0]+6,uv_wR[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,0),2)
    if uv_tL: cv2.circle(vis, uv_tL, 7, (0,0,255), -1); cv2.putText(vis,'L_tip',(uv_tL[0]+6,uv_tL[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),2)
    if uv_tR: cv2.circle(vis, uv_tR, 7, (255,0,0), -1); cv2.putText(vis,'R_tip',(uv_tR[0]+6,uv_tR[1]-6),cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,0,0),2)

    cv2.putText(vis, f'frame {fr}', (20,40), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0,255,0), 2)
    out_path = str(Path(OUT_DIR_TEST)/f'test_{fr}.png')
    cv2.imwrite(out_path, vis)
    print('Saved:', out_path)

print('Done.')


Saved: test_vis/test_50.png
Saved: test_vis/test_75.png
Saved: test_vis/test_125.png
Saved: test_vis/test_225.png
Saved: test_vis/test_275.png
Saved: test_vis/test_325.png
Saved: test_vis/test_425.png
Saved: test_vis/test_525.png
Saved: test_vis/test_625.png
Saved: test_vis/test_725.png
Saved: test_vis/test_825.png
Saved: test_vis/test_925.png
Saved: test_vis/test_1025.png
Saved: test_vis/test_1125.png
Saved: test_vis/test_1225.png
Saved: test_vis/test_1325.png
Saved: test_vis/test_1425.png
Saved: test_vis/test_1525.png
Saved: test_vis/test_1625.png
Done.
